根据CAN分析文件，进行数据文件切割

In [2]:
import pandas as pd

# 导入0003.csv文件
df1 = pd.read_csv('20250214.csv', skiprows=1)
# df1 = pd.read_csv('20250117_0003.csv', skiprows=1)

# 导入0004.csv文件2
df2 = pd.read_csv('20250214can.csv')
# df2 = pd.read_csv('20250117_0004.csv')

# 根据“Data”列创建标签
df2['label'] = df2['Data'].apply(lambda x: 1 if x[-2:] == '10' else 0)


# 打印数据框的前几行以验证导入是否成功
print(df1.head())
print(df2.head())

      (ms)       (V)
0  0.00000  0.000000
1  0.00008  0.042753
2  0.00016  0.000000
3  0.00024  0.042753
4  0.00032  0.042753
   Buffer Number  Packet  Start Time  End Time        ID  RTR  FDF  DLC  \
0              1       1    0.006250  0.006536  18F40117    0    0    8   
1              1       2    0.012534  0.012818  18F40117    0    0    8   
2              1       3    0.025643  0.025929  18F40117    0    0    8   
3              1       4    0.044890  0.045176  18F40117    0    0    8   
4              1       5    0.062533  0.062817  18F40117    0    0    8   

                      Data    CRC  ACK  CRC Valid  Bit Stuffing Valid  Valid  \
0  00 00 00 00 00 00 00 10  1B 81    0          1                   1      1   
1  00 00 00 00 00 00 00 00  36 16    0          1                   1      1   
2  00 00 00 00 00 00 00 10  1B 81    0          1                   1      1   
3  00 00 00 00 00 00 00 10  1B 81    0          1                   1      1   
4  00 00 00 00 00 00 00

In [3]:
type(df2['Data'][0])

str

In [ ]:
# 示例输入
id_hex = "18EFFFFF"  # 16进制ID值（包含标准ID、扩展ID、SRR、IDE和RTR）
frame_hex = "0102030405060708"  # 数据字段（8字节）
rtr = 0  # RTR位
fdf = 1  # FDF位（1表示扩展帧）
dlc = 8  # 数据长度码（8字节）
crc = "18EF"

# 将16进制转换为二进制比特流
def hex_to_bits(hex_str, length=None):
    if length:
        return bin(int(hex_str, 16))[2:].zfill(length)
    return bin(int(hex_str, 16))[2:].zfill(len(hex_str) * 4)

# 计算填充位数
def calculate_stuffing_bits(bits):
    stuffing_count = 0
    count = 0
    prev_bit = None
    for bit in bits:
        if bit == prev_bit:
            count += 1
            if count == 5:
                stuffing_count += 1
                count = 0
        else:
            count = 0
        prev_bit = bit
    return stuffing_count

# 扩充id二进制值
def insert_bits(binary_str, insert_value):
    # 确保输入是一个二进制字符串
    if not all(bit in '01' for bit in binary_str):
        raise ValueError("输入必须是一个二进制字符串")
    
    # 在第12和第13位插入1
    modified_str = binary_str[:11] + '11' + binary_str[11:]
    
    # 在末尾插入给定值
    modified_str += str(insert_value)
    
    return modified_str

# 解析扩展帧
def parse_extended_frame(id_hex, frame_hex, dlc, rtr, fdf, crc):
    # 将ID转换为二进制
    id_bits = hex_to_bits(id_hex, length=29)  # 29位（11位标准ID + 18位扩展ID）

    id_bits_all = insert_bits(id_bits, rtr)   # 32位（11位标准ID + 18位扩展ID + 3位控制）

    # 控制字段（IDE=1表示扩展帧，r0=0，DLC）
    control_bits = str(fdf) + "0" + bin(dlc)[2:].zfill(4)  # 6位

    # 数据字段
    data_bits = hex_to_bits(frame_hex)  # DLC × 8位

    # CRC字段（假设为15位，实际需要计算）
    crc_bits = hex_to_bits(crc).zfill(4)  # 占位符

    # 组合完整帧
    full_frame_bits = id_bits_all + control_bits + data_bits + crc_bits

    # 计算填充位数
    stuffing_bits = calculate_stuffing_bits(full_frame_bits)

    # 计算RTR和CRC的位置
    rtr_index = 32 + calculate_stuffing_bits(id_bits)  # RTR位索引
    crc_end_index = rtr_index + len(control_bits) + len(data_bits) + 15 + calculate_stuffing_bits(control_bits + data_bits + crc_bits)

    return rtr_index, crc_end_index

# 计算RTR和CRC的位置
rtr_index, crc_end_index = parse_extended_frame(id_hex, frame_hex, dlc, rtr, fdf, crc)
print(f"RTR起始位索引: {rtr_index}")
print(f"CRC结束位索引: {crc_end_index}")


<class 'str'>
RTR起始位索引: 35
CRC结束位索引: 126


In [31]:
# df1['时间'] = pd.to_numeric(df1['时间'])

# 将第一个文件的时间列从毫秒转换为秒
df1['时间（s）'] = df1['(ms)'] / 1000

# 创建一个空的列表来存储切割后的DataFrame
dataframes = []

# 遍历第二个文件中的每一行，获取start time和end time
for index, row in df2.iterrows():
    start_time = row['Start Time']
    end_time = row['End Time']
    label = row['label']
    
    # 根据start time和end time切割第一个文件的数据
    mask = (df1['时间（s）'] >= start_time) & (df1['时间（s）'] <= end_time)
    sliced_df = df1[mask]
    sliced_df['label'] = label

    id_hex = row['ID'].replace(" ", "")  # 11位标准ID + 18位扩展ID
    frame_hex = row['Data'].replace(" ", "")  # 数据字段
    rtr = row['RTR']  # RTR位
    fdf = row['FDF']  # FDF位
    crc = row['CRC'].replace(" ", "")  # CRC值
    dlc = row['DLC']  # 数据长度码（8字节）

    rtr_index, crc_end_index = parse_extended_frame(id_hex, frame_hex, dlc, rtr, fdf, crc)

    # 进一步裁剪 DataFrame 中给定行数范围内的行
    further_sliced_df = sliced_df.iloc[(int(rtr_index)*25):(int(crc_end_index)*25)]

    
    # print(sliced_df.iloc[int(rtr_index):int(crc_end_index)])
    # sliced_df.iloc[rtr_index, crc_end_index]
    # 将切割后的DataFrame添加到列表中
    if (further_sliced_df['(V)'] > 2.5).any():
        continue
    dataframes.append(further_sliced_df)

# 打印切割后的DataFrame以验证结果
for i, df in enumerate(dataframes):
    # if i==0:
    #     df.to_csv('dataorigin1.csv', index=True)
    print(f"DataFrame {i+1}:")
    print(df)

C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['label'] = label
C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['label'] = label
C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['label'] = label
C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['label'] = label
C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['label'] = label
C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['label'] = label
C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

...

C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['label'] = label
C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['label'] = label
C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['label'] = label
C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['label'] = label
C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
DataFrame 1:
          (ms)       (V)     时间（s）  label
78953  6.31624  2.142440  0.006316      1
78954  6.31632  2.142440  0.006316      1
78955  6.31640  2.228100  0.006316      1
78956  6.31648  2.142440  0.006316      1
78957  6.31656  2.099686  0.006317      1
...        ...       ...       ...    ...
81423  6.51384  0.171321  0.006514      1
81424  6.51392  0.128414  0.006514      1
81425  6.51400  0.042753  0.006514      1
81426  6.51408 -0.042753  0.006514      1
81427  6.51416  0.000000  0.006514      1

[2475 rows x 4 columns]
DataFrame 2:
            (ms)       (V)     时间（s）  label
321366  25.70928  2.099686  0.025709      1
321367  25.70936  2.185347  0.025709      1
321368  25.70944  2.185347  0.025709      1
321369  25.70952  2.142440  0.025710      1
3

C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['label'] = label
C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sliced_df['label'] = label
C:\Users\richa\AppData\Local\Temp\ipykernel_31820\1144470430.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

上面将数据裁剪为rtr到crc段内的部分。这里清楚一下不规则片段。

In [ ]:
# 对每个DataFrame进行进一步切割，从第800行开始，到2925行结束
# further_cut_dataframes = []
# for df in dataframes:
#     further_cut_df = df.iloc[800:2975].copy()
#     further_cut_dataframes.append(further_cut_df)

# # 打印进一步切割后的DataFrame以验证结果
# for i, df in enumerate(further_cut_dataframes):
#     print(f"Further Cut DataFrame {i+1}:")
#     print(df)

# # 将所有进一步切割后的DataFrame合并为一个DataFrame
# combined_df = pd.concat(further_cut_dataframes, ignore_index=True)

# # 将合并后的DataFrame保存为文件
# combined_df.to_csv('statistics.csv', index=False)

# # 打印合并后的DataFrame
# print(combined_df)

Further Cut DataFrame 1:
          (ms)       (V)     时间（s）  label
78928  6.31424  0.042753  0.006314      1
78929  6.31432  0.042753  0.006314      1
78930  6.31440  0.000000  0.006314      1
78931  6.31448  0.042753  0.006314      1
78932  6.31456  0.042753  0.006315      1
...        ...       ...       ...    ...
81098  6.48784  0.299889  0.006488      1
81099  6.48792  0.128414  0.006488      1
81100  6.48800  0.042753  0.006488      1
81101  6.48808  0.000000  0.006488      1
81102  6.48816 -0.085661  0.006488      1

[2175 rows x 4 columns]
Further Cut DataFrame 2:
            (ms)       (V)     时间（s）  label
157472  12.59776  0.042753  0.012598      0
157473  12.59784  0.042753  0.012598      0
157474  12.59792  0.042753  0.012598      0
157475  12.59800  0.085661  0.012598      0
157476  12.59808  0.042753  0.012598      0
...          ...       ...       ...    ...
159642  12.77136  0.085661  0.012771      0
159643  12.77144  0.042753  0.012771      0
159644  12.77152  0.04275

In [32]:
def detect_edges_and_split(df):
    segments = []
    i = 0
    while i < len(df) - 1:
        prev_voltage = df.iloc[i - 1]['(V)'] if i > 0 else None
        curr_voltage = df.iloc[i]['(V)']
        next_voltage = df.iloc[i + 1]['(V)'] if i < len(df) - 1 else None
        
        # 检测上升沿开始
        if prev_voltage is not None and prev_voltage < 0.10 and curr_voltage > 0.10:
            start_idx = i-1
            while i < len(df) - 1 and not (curr_voltage > 2.0 and next_voltage > 2.0):
                if curr_voltage < 0.15:
                    # 如果上升沿过程中出现小于0.15的值，跳过当前大循环
                    break
                i += 1
                curr_voltage = df.iloc[i]['(V)']
                next_voltage = df.iloc[i + 1]['(V)'] if i < len(df) - 1 else None

            if next_voltage == None:
                break
            
            if curr_voltage < 0.15:
                    # 如果上升沿过程中出现小于0.15的值，跳过当前大循环
                i += 1
                continue

            end_rising_idx = i + 2
            i += 1

            
            # 检测稳定态
            while i < len(df) - 1 and not (curr_voltage > 2.0 and next_voltage < 2.0):
                i += 1
                curr_voltage = df.iloc[i]['(V)']
                next_voltage = df.iloc[i + 1]['(V)'] if i < len(df) - 1 else None
            start_falling_idx = i
            
            if i==len(df)-1:
                break

            # 检测下降沿结束
            while i < len(df) - 1 and not (curr_voltage < 0.15 and prev_voltage > 0.15):
                i += 1
                prev_voltage = df.iloc[i - 1]['(V)']
                curr_voltage = df.iloc[i]['(V)']
            end_falling_idx = i + 1
            
            # 将上升沿开始到下降沿结束的数据作为一个独立的段
            segment = df.iloc[start_idx:end_falling_idx][['(V)', '(ms)', 'label']]
            # print(start_idx)
            # print(end_rising_idx)
            segment['state'] = 'steady'  # 默认标记为稳态
            segment.iloc[0:(end_rising_idx - start_idx), segment.columns.get_loc('state')] = 'rising'
            # print(start_falling_idx)
            # print(end_falling_idx)
            segment.iloc[(start_falling_idx - start_idx):(end_falling_idx - start_idx), segment.columns.get_loc('state')] = 'falling'
            
            
            segments.append(segment)
        
        i += 1
    
    return segments

# 对每个切割后的DataFrame进行处理
all_segments = []
for i, df in enumerate(dataframes):
    segments = detect_edges_and_split(df)
    all_segments.extend(segments)



In [33]:
from scipy.stats import skew, kurtosis
import numpy as np
from scipy.fft import fft, fftfreq
# 打印每个段以验证结果
for i, segment in enumerate(all_segments):
    if i<53:
        print(f"Segment {i+1}:")
        print(segment)
        
        df = segment[0:3]
        # df = segment[segment['state'] == 'rising']
        fft_values = fft(df['(V)'].values)
        fft_freq = fftfreq(len(df), d=(df['(ms)'].diff().mean() / 1000))  # 将时间间隔转换为秒
        fft_magnitude = np.abs(fft_values)

        # 只取前一半的结果，忽略负值部分
        half_n =  (len(fft_values) + 1) // 2
        fft_values = fft_values[:half_n]
        fft_freq = fft_freq[:half_n]
        fft_magnitude = fft_magnitude[:half_n]
        # rising_edge = segment[segment['state'] == 'rising']
        # print(rising_edge)
        kurt = kurtosis(fft_magnitude)
        skewness = skew(fft_magnitude)
        print(kurt)
        print(skewness)
        # break

Segment 1:
            (V)     (ms)  label    state
79049  0.042753  6.32392      1   rising
79050  1.499754  6.32400      1   rising
79051  2.013872  6.32408      1   rising
79052  2.099686  6.32416      1   rising
79053  2.099686  6.32424      1   steady
...         ...      ...    ...      ...
79174  2.099686  6.33392      1  falling
79175  0.728347  6.33400      1  falling
79176  0.299889  6.33408      1  falling
79177  0.171321  6.33416      1  falling
79178  0.085661  6.33424      1  falling

[130 rows x 4 columns]
-2.0
7.802555085514782e-16
Segment 2:
            (V)     (ms)  label    state
79199  0.042753  6.33592      1   rising
79200  1.799643  6.33600      1   rising
79201  2.056779  6.33608      1   rising
79202  2.142440  6.33616      1   rising
79203  2.099686  6.33624      1   steady
...         ...      ...    ...      ...
79323  2.099686  6.34584      1  falling
79324  1.971118  6.34592      1  falling
79325  0.514118  6.34600      1  falling
79326  0.256982  6.34608 

In [8]:
print(segment.loc[563248 :563260])

             (V)      (ms)  label   state
563248  0.042753  45.05984      1  rising
563249  0.171321  45.05992      1  rising
563250  1.928211  45.06000      1  rising
563251  2.056779  45.06008      1  rising
563252  2.099686  45.06016      1  rising
563253  2.099686  45.06024      1  steady
563254  2.142440  45.06032      1  steady
563255  2.142440  45.06040      1  steady
563256  2.099686  45.06048      1  steady
563257  2.099686  45.06056      1  steady
563258  2.056779  45.06064      1  steady
563259  2.056779  45.06072      1  steady
563260  2.142440  45.06080      1  steady


In [ ]:
# from scipy.fft import fft, fftfreq
# rising_edge = segment[0:4]
# dfe = rising_edge
# fft_values = fft(rising_edge['(V)'].values)
# fft_magnitude = np.abs(fft_values)
# print(fft_magnitude)
# print(dfe['(ms)'].diff().mean() / 1000)
# fft_freq = fftfreq(len(dfe), d=(dfe['(ms)'].diff().mean() / 1000))
# print(fft_freq)
# centroid = np.sum(fft_freq * fft_magnitude) / np.sum(fft_magnitude)

# print(centroid)

NameError: name 'np' is not defined

In [34]:
from scipy.stats import skew, kurtosis
import numpy as np
from scipy.fft import fft, fftfreq

# 定义一个函数来计算统计特征
def calculate_statistics(df):
    stats = {
        'mean': df['(V)'].mean(),
        'std': df['(V)'].std(),
        'mean_diff': df['(V)'].diff().mean(),
        'skew': skew(df['(V)']),
        'kurtosis': kurtosis(df['(V)']),
        'rms': np.sqrt(np.mean(df['(V)']**2)),
        'max': df['(V)'].max(),
        'min': df['(V)'].min()
    }
    return stats

# 定义一个函数来计算频域统计特征
def calculate_frequency_domain_statistics(df):
    # 进行傅里叶变换
    fft_values = fft(df['(V)'].values)
    fft_freq = fftfreq(len(df), d=(df['(ms)'].diff().mean() / 1000))  # 将时间间隔转换为秒
    fft_magnitude = np.abs(fft_values)

    # 只取前一半的结果，忽略负值部分
    half_n =  (len(fft_values) + 1) // 2
    fft_values = fft_values[:half_n]
    fft_freq = fft_freq[:half_n]
    fft_magnitude = fft_magnitude[:half_n]

    # 过滤掉零值
    non_zero_indices = fft_magnitude > 0
    fft_magnitude_entro = fft_magnitude[non_zero_indices]
    # fft_freq = fft_freq[non_zero_indices]
    
    # 计算频域统计特征
    centroid = np.sum(fft_freq * fft_magnitude) / np.sum(fft_magnitude)
    entropy = -np.sum((fft_magnitude_entro / np.sum(fft_magnitude_entro)) * np.log(fft_magnitude_entro / np.sum(fft_magnitude_entro)))
    spread = np.sqrt(np.sum(((fft_freq - centroid) ** 2) * fft_magnitude) / np.sum(fft_magnitude))
    skewness = skew(fft_magnitude)
    avg_value = np.mean(fft_magnitude)
    variance = np.var(fft_magnitude)
    kurt = kurtosis(fft_magnitude)
    irregularity = np.sum(np.abs(np.diff(fft_magnitude)))
    
    stats = {
        'fft_centroid': centroid,
        'fft_entropy': entropy,
        'fft_spread': spread,
        'fft_skewness': skewness,
        'fft_avg_value': avg_value,
        'fft_variance': variance,
        'fft_kurtosis': kurt,
        'fft_irregularity': irregularity
    }
    return stats

# 创建一个空的列表来存储统计特征
statistics = []

# 对每个段进行处理
for segment in all_segments:
    label = segment['label'].iloc[0]  # 获取第一个label值
    rising_edge = segment[segment['state'] == 'rising']
    falling_edge = segment[segment['state'] == 'falling']
    steady_state = segment[segment['state'] == 'steady']
    
    rising_time_stats = calculate_statistics(rising_edge)
    falling_time_stats = calculate_statistics(falling_edge)
    steady_time_stats = calculate_statistics(steady_state)

    rising_freq_stats = calculate_frequency_domain_statistics(rising_edge)
    falling_freq_stats = calculate_frequency_domain_statistics(falling_edge)
    steady_freq_stats = calculate_frequency_domain_statistics(steady_state)
    
    combined_stats = {
        'label': label,
        'rising_mean': rising_time_stats['mean'],
        'rising_std': rising_time_stats['std'],
        'rising_mean_diff': rising_time_stats['mean_diff'],
        'rising_skew': rising_time_stats['skew'],
        'rising_kurtosis': rising_time_stats['kurtosis'],
        'rising_rms': rising_time_stats['rms'],
        'rising_max': rising_time_stats['max'],
        'rising_min': rising_time_stats['min'],
        'rising_fft_centroid': rising_freq_stats['fft_centroid'],
        'rising_fft_entropy': rising_freq_stats['fft_entropy'],
        'rising_fft_spread': rising_freq_stats['fft_spread'],
        'rising_fft_skewness': rising_freq_stats['fft_skewness'],
        'rising_fft_avg_value': rising_freq_stats['fft_avg_value'],
        'rising_fft_variance': rising_freq_stats['fft_variance'],
        'rising_fft_kurtosis': rising_freq_stats['fft_kurtosis'],
        'rising_fft_irregularity': rising_freq_stats['fft_irregularity'],
        'falling_mean': falling_time_stats['mean'],
        'falling_std': falling_time_stats['std'],
        'falling_mean_diff': falling_time_stats['mean_diff'],
        'falling_skew': falling_time_stats['skew'],
        'falling_kurtosis': falling_time_stats['kurtosis'],
        'falling_rms': falling_time_stats['rms'],
        'falling_max': falling_time_stats['max'],
        'falling_min': falling_time_stats['min'],
        'falling_fft_centroid': falling_freq_stats['fft_centroid'],
        'falling_fft_entropy': falling_freq_stats['fft_entropy'],
        'falling_fft_spread': falling_freq_stats['fft_spread'],
        'falling_fft_skewness': falling_freq_stats['fft_skewness'],
        'falling_fft_avg_value': falling_freq_stats['fft_avg_value'],
        'falling_fft_variance': falling_freq_stats['fft_variance'],
        'falling_fft_kurtosis': falling_freq_stats['fft_kurtosis'],
        'falling_fft_irregularity': falling_freq_stats['fft_irregularity'],
        'steady_mean': steady_time_stats['mean'],
        'steady_std': steady_time_stats['std'],
        'steady_mean_diff': steady_time_stats['mean_diff'],
        'steady_skew': steady_time_stats['skew'],
        'steady_kurtosis': steady_time_stats['kurtosis'],
        'steady_rms': steady_time_stats['rms'],
        'steady_max': steady_time_stats['max'],
        'steady_min': steady_time_stats['min'],
        'steady_fft_centroid': steady_freq_stats['fft_centroid'],
        'steady_fft_entropy': steady_freq_stats['fft_entropy'],
        'steady_fft_spread': steady_freq_stats['fft_spread'],
        'steady_fft_skewness': steady_freq_stats['fft_skewness'],
        'steady_fft_avg_value': steady_freq_stats['fft_avg_value'],
        'steady_fft_variance': steady_freq_stats['fft_variance'],
        'steady_fft_kurtosis': steady_freq_stats['fft_kurtosis'],
        'steady_fft_irregularity': steady_freq_stats['fft_irregularity']
    }
    
    statistics.append(combined_stats)

# 将统计特征转换为DataFrame
stats_df = pd.DataFrame(statistics)

# 保存统计特征DataFrame为文件
stats_df.to_csv('statistics1.csv', index=False)

# 打印统计特征DataFrame
print(stats_df)

     label  rising_mean  rising_std  rising_mean_diff  rising_skew  \
0        1     1.414016    0.951785          0.685644    -0.910509   
1        1     1.510404    0.989217          0.699896    -1.082147   
2        1     1.276913    1.051132          0.514233    -0.412523   
3        1     1.474040    0.878513          0.503506    -0.869938   
4        1     1.414016    0.951785          0.685644    -0.910509   
..     ...          ...         ...               ...          ...   
836      1     1.510404    0.952619          0.657039    -1.134755   
837      1     1.319728    0.982528          0.503506    -0.476227   
838      1     1.396869    0.911853          0.503506    -0.691688   
839      1     1.491142    0.874760          0.514233    -1.056565   
840      1     1.456885    0.966475          0.685644    -0.998282   

     rising_kurtosis  rising_rms  rising_max  rising_min  rising_fft_centroid  \
0          -0.895245    1.636723    2.099686    0.042753         8.344155e+05 

In [35]:
# 查找包含 NaN 值的行
nan_rows = stats_df[pd.isna(stats_df).any(axis=1)]

# 打印包含 NaN 值的行
# print("存在 NaN 的行：")
# print(nan_rows)
# 打印这些行中只包含NaN的列
for index, row in nan_rows.iterrows():
    nan_columns = row[row.isna()].index
    print(f"Row {index} has NaN in columns: {nan_columns.tolist()}")

In [36]:
print(nan_rows['rising_fft_skewness'])

Series([], Name: rising_fft_skewness, dtype: float64)


这里开始都是之前废稿
import pandas as pd

# 定义一个函数来检测上升沿、下降沿和稳定态
def detect_edges(df):
    rising_edge = []
    falling_edge = []
    steady_state = []

    i = 0
    while i < len(df) - 1:
        prev_voltage = df.iloc[i - 1]['(V)'] if i > 0 else None
        curr_voltage = df.iloc[i]['(V)']
        next_voltage = df.iloc[i + 1]['(V)'] if i < len(df) - 1 else None
        
        # 检测上升沿
        if prev_voltage is not None and prev_voltage < 0.1 and curr_voltage > 0.1:
            start_idx = i
            while i < len(df) - 1 and not (curr_voltage > 2.0 and next_voltage > 2.0):
                i += 1
                curr_voltage = df.iloc[i]['(V)']
                next_voltage = df.iloc[i + 1]['(V)'] if i < len(df) - 1 else None
            rising_edge.extend(df.iloc[start_idx:i+1][['(V)', '(ms)']].values.tolist())
        
        # 检测下降沿
        elif curr_voltage > 2.0 and next_voltage < 2.0:
            start_idx = i
            while i < len(df) - 1 and not (curr_voltage < 0.1 and prev_voltage > 0.1):
                i += 1
                prev_voltage = df.iloc[i - 1]['(V)']
                curr_voltage = df.iloc[i]['(V)']
            falling_edge.extend(df.iloc[start_idx:i+1][['(V)', '(ms)']].values.tolist())
        
        # 其余部分是稳定态
        else:
            steady_state.append(df.iloc[i][['(V)', '(ms)']].values.tolist())
        
        i += 1
    
    rising_edge_df = pd.DataFrame(rising_edge, columns=['(V)', '(ms)'])
    falling_edge_df = pd.DataFrame(falling_edge, columns=['(V)', '(ms)'])
    steady_state_df = pd.DataFrame(steady_state, columns=['(V)', '(ms)'])
    
    return rising_edge_df, falling_edge_df, steady_state_df


# 对每个切割后的DataFrame进行处理
for i, df in enumerate(dataframes):
    rising_edge, falling_edge, steady_state = detect_edges(df)
    print(f"DataFrame {i+1} 上升沿:")
    print(rising_edge)
    print(f"DataFrame {i+1} 下降沿:")
    print(falling_edge)
    print(f"DataFrame {i+1} 稳定态:")
    print(steady_state)
    print(f"DataFrame {i+1} 标签:")
    print(df['label'].iloc[0])
    break
